# Wind Turbine Farm Management Assistant

## Objective

This notebook demonstrates the following:

- Utilizing Assistant tools such as the Code Interpreter and Function calling, this bot is capable of retrieving a CSV file that illustrates turbine wind speed, voltage, and the last maintenance date. It assists you in reviewing the file contents and aids in determining whether a specific turbine is in need of maintenance.

This tutorial uses the following Azure AI services:
- Access to Azure OpenAI Service - you can apply for access [here](https://aka.ms/oai/access)
- Azure OpenAI service - you can create it from instructions [here](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/create-resource)
- Azure OpenAI Studio - go to [https://oai.azure.com/](https://oai.azure.com/) to work with the Assistants API Playground
- A connection to the Azure OpenAI Service with a [Key and Endpoint](https://learn.microsoft.com/en-us/azure/ai-services/openai/chatgpt-quickstart)

Reference:
- Learn more about how to use Assistants with our [How-to guide on Assistants](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/assistant).
- [Assistants OpenAI Overview](https://platform.openai.com/docs/assistants/overview)


## Time

You should expect to spend 5-10 minutes running this sample.

## About this example

This sample demonstrates the creation of an Azure OpenAI Assistant named "Turbine Management Assistant" utilizing the Azure OpenAI API. The assistant functions as a management tool for wind turbine farms, providing data, charts, and actionable insights regarding the status and performance of each turbine within the farm. The notebook/script orchestrates a dialogue with the assistant, navigating through different inquiries and scenarios to demonstrate its capabilities.

### Data
This sample uses files from the folder [`data/`](./data/) in this repo. You can clone this repo or copy this folder to make sure you have access to these files when running the sample.


## Before you begin



### Installation

Install the following packages required to execute this notebook. 



In [5]:
# Install the packages
%pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


### Parameters

In [6]:
import os

from dotenv import load_dotenv

load_dotenv("../.env")

api_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("OPENAI_VERSION")
api_deployment_name = os.getenv("OPENAI_GPT_DEPLOYMENT")
email_URI = os.getenv("EMAIL_URI")

should_cleanup: bool = False

## Run this Example

### Load the required libraries

In [7]:
import io
import time
from datetime import datetime
from pathlib import Path
from typing import Iterable

from openai import AzureOpenAI
from openai.types import FileObject
from openai.types.beta.threads.message_content_image_file import MessageContentImageFile
from openai.types.beta.threads.message_content_text import MessageContentText
from openai.types.beta.threads.messages import MessageFile
from PIL import Image

### Create an AzureOpenAI client

In [9]:
client = AzureOpenAI(api_key=api_key, api_version=api_version, azure_endpoint=api_endpoint)

### Define the Assistant tools

In [10]:
tools_list = [
    {"type": "code_interpreter"},
]

### Upload the Assistant file(s)

In [14]:
DATA_FOLDER = "data/"


def upload_file(client: AzureOpenAI, path: str) -> FileObject:
    with Path(path).open("rb") as f:
        return client.files.create(file=f, purpose="assistants")


arr = os.listdir(DATA_FOLDER)

print(arr)
assistant_files = []
for file in arr:
    filePath = DATA_FOLDER + file
    assistant_files.append(upload_file(client, filePath))

file_ids = [file.id for file in assistant_files]

print(file_ids)



['turbines.csv']
['assistant-oFb1SVUJGH2ArOFdUOD5xWBW']


### Create an Assistant and a Thread

In [15]:
assistant = client.beta.assistants.create(
    name="Portfolio Management Assistant",
    instructions="You are an assistant that can help manage wind turbine farm. "
    + "The turbines operating ranges are output voltages of 33kv-35kv and RPM of 15-25. Wind speed is measured in miles per hour."
    + "Maintenance should occur every 12 months. Greet the user by saying, 'Welcome Turbine Management Assistant.'",
    tools=tools_list,
    model=api_deployment_name,
    file_ids=file_ids,
)

thread = client.beta.threads.create()

### Format and display the Assistant Messages for text and image

In [16]:
def format_messages(messages: Iterable[MessageFile]) -> None:
    message_list = []

    # Get all the messages till the last user message
    for message in messages:
        message_list.append(message)
        if message.role == "user":
            break

    # Reverse the messages to show the last user message first
    message_list.reverse()

    # Print the user or Assistant messages or images
    for message in message_list:
        for item in message.content:
            # Determine the content type
            if isinstance(item, MessageContentText):
                print(f"{message.role}:\n{item.text.value}\n")
            elif isinstance(item, MessageContentImageFile):
                # Retrieve image from file id
                response_content = client.files.content(item.image_file.file_id)
                data_in_bytes = response_content.read()
                # Convert bytes to image
                readable_buffer = io.BytesIO(data_in_bytes)
                image = Image.open(readable_buffer)
                # Resize image to fit in terminal
                width, height = image.size
                image = image.resize((width // 2, height // 2), Image.LANCZOS)
                # Display image
                image.show()

### Process the user messages

In [17]:
def process_message(content: str) -> None:
    client.beta.threads.messages.create(thread_id=thread.id, role="user", content=content)

    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions="The current date and time is: " + datetime.now().strftime("%x %X") + ". ",
    )

    print("processing...")
    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run.status == "completed" or run.status == "failed":
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            format_messages(messages)
            break
        if run.status == "expired":
            # Handle expired
            break
        if run.status == "cancelled":
            # Handle cancelled
            break
        if run.status == "requires_action":
            pass
        else:
            # print("in progress...",run.status)
            time.sleep(5)

###  Have a conversation with the Assistant

In [18]:
process_message("What is the status of turbine 1001 and 1003?")

processing...
user:
What is the status of turbine 1001 and 1003?

assistant:
To provide you with the status of turbines 1001 and 1003, I need to check the contents of the file you've uploaded. First, I'll examine the file to determine its format and then extract the relevant information related to the status of turbines 1001 and 1003.

Let's start by examining the file.

assistant:
The file appears to be in CSV format and contains a header row with the following columns: Turbine_ID, Wind_Speed, RPM, Voltage, and Maintenance_Date. I'll go ahead and read the entire file to search for information regarding the status of turbines 1001 and 1003.

assistant:
I have loaded the data, and it includes information on different turbines. To find the status of turbines 1001 and 1003, I will filter the data for these two turbine IDs and present the relevant information. Let's proceed with that.

assistant:
The status of turbines 1001 and 1003 is as follows:

- Turbine 1001:
  - Wind Speed: 32
  - RP

In [19]:
process_message("Generate a chart of the normal operating ranges?")

processing...
user:
Generate a chart of the normal operating ranges?

assistant:
To generate a chart of normal operating ranges for the turbines, we need to define what the "normal" operating range is for each parameter (Wind Speed, RPM, and Voltage). If the normal operating ranges are specified in the data, we could use that information; otherwise, we would need external information to set those ranges. 

Since I don't have external information on the normal operating ranges and the uploaded file does not contain this information, I can proceed in two ways: 
1. Define arbitrary normal operating ranges based on the data we have, although this won't necessarily reflect the actual normal operating conditions.
2. If you can provide the normal operating ranges for each of the parameters, I can create a chart reflecting those ranges.

Please advise on how to proceed, or provide the normal operating ranges for Wind Speed, RPM, and Voltage.



In [ ]:
process_message(
    "What turbines are operating outside normal ranges?\n\n"
    + 'Sample: [{"turbine":"","voltage":30,"RPM":15,"reason":"Voltage outside of normal range."},\n'
    + '{"turbine":"","voltage":30,"rpm":10,"reason":"RPM and Voltage outside of normal ranges."},\n'
    + '{"turbine":"","voltage":33,"rpm":5,"reason":"RPM outside of normal range."}]\n\n'
    + 'Output format: [{"turbine":"","voltage":0,"RPM":0,"reason":""}]\n\n'
    + "Output in JSON format only."
)

In [20]:
process_message("What turbines need maintenance?")

processing...
user:
What turbines need maintenance?

assistant:
To determine which turbines need maintenance, we need some criteria. Typically, this could be based on:

1. A specific maintenance schedule (e.g., maintenance is required every 6 months).
2. The actual performance data (e.g., RPM, Voltage) falling outside normal operating ranges.
3. The date of the last maintenance relative to the current date.

The file includes a "Maintenance_Date" which could be the date when maintenance is scheduled. If we assume that the date provided in the "Maintenance_Date" column is the next scheduled maintenance date, we can compare that date to the current date to determine which turbines are due for maintenance.

Let's identify any turbines with a maintenance date that is in the past relative to today's date (03/11/24).

assistant:
Based on the assumption that the "Maintenance_Date" column represents the next scheduled maintenance date and comparing it with today's date (11th March 2024), the f

## Cleaning up


In [ ]:
if should_cleanup:
    client.beta.assistants.delete(assistant.id)
    client.beta.threads.delete(thread.id)
    for file in assistant_files:
        client.files.delete(file.id)